assignment3

In [16]:
%matplotlib inline

In [17]:
import csv
import random

import numpy as np
import pandas as pd
import networkx as nx

## Create dataframe

In [18]:
import pandas as pd

# Read the CSV file into a DataFrame
csv_file_path = 'tiktok.csv'  # Replace 'your_file.csv' with the actual file path
df = pd.read_csv(csv_file_path)

# Specify the columns you want to keep
desired_columns = ['account_id', 'followers', 'following', 'likes', 'videos_count']  # Replace with your actual column names

# Create a new DataFrame with only the desired columns
df_selected_columns = df[desired_columns]

df_selected_columns.set_index('account_id', inplace=True)

# Print or use the new DataFrame as needed
df_selected_columns

,followers,following,likes,videos_count
account_id,,,,
a.r.m.y664,1755,1443,1586,20
a2_9r,354,1422,3843,225
ti.po.check,1960,243,24000,228
nutrifersanort,43,41,100,5
bhgdee,276,2262,126,4
...,...,...,...,...
lpk.jp,956,294,4028,110
loup_32,55,226,101,4
djbigkev,12100,256,102400,89


## Apply L1 Normalization

In [19]:
columns_to_normalize = df_selected_columns.columns[1:]

df_selected_columns[columns_to_normalize] = df_selected_columns[columns_to_normalize].apply(pd.to_numeric, errors='coerce')

df_selected_columns[columns_to_normalize] = df_selected_columns[columns_to_normalize].fillna(df_selected_columns[columns_to_normalize].mean())

df_norm = df_selected_columns[columns_to_normalize].divide(df_selected_columns[columns_to_normalize].abs().sum(axis=1), axis=0)

print(df_norm.head(10))


                   following     likes  videos_count
account_id                                          
a.r.m.y664          0.473270  0.520171      0.006560
a2_9r               0.259016  0.700000      0.040984
ti.po.check         0.009930  0.980753      0.009317
nutrifersanort      0.280822  0.684932      0.034247
bhgdee              0.945652  0.052676      0.001672
preyah_             0.000192  0.999577      0.000231
anastasiaashley     0.000030  0.999894      0.000076
_sanek__56rus       0.629598  0.354455      0.015947
_rinm               0.745902  0.213115      0.040984
_k_1_ji_ji_e_ip__   0.846715  0.131387      0.021898


/var/folders/m_/k8tcd9ts10v8f132ksgxjks00000gp/T/ipykernel_47695/1125138806.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected_columns[columns_to_normalize] = df_selected_columns[columns_to_normalize].apply(pd.to_numeric, errors='coerce')
/var/folders/m_/k8tcd9ts10v8f132ksgxjks00000gp/T/ipykernel_47695/1125138806.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected_columns[columns_to_normalize] = df_selected_columns[columns_to_normalize].fillna(df_selected_columns[columns_to_normaliz

In [20]:
# check that all add up to 1, meaning normalization is correct

df_norm.sum(axis=1).head(20)

account_id
a.r.m.y664            1.0
a2_9r                 1.0
ti.po.check           1.0
nutrifersanort        1.0
bhgdee                1.0
preyah_               1.0
anastasiaashley       1.0
_sanek__56rus         1.0
_rinm                 1.0
_k_1_ji_ji_e_ip__     1.0
xrb_010               1.0
soyon_on_fire         1.0
laurenwolfe           1.0
mybeautifulfantasy    1.0
itshoneydone          1.0
curlyheadkay3         1.0
ryanmarkparsons       1.0
a3536363773           1.0
a.oakley03            1.0
_lipsticklinz         1.0
dtype: float64

In [21]:
# find user with highest 'following'

user_with_highest_following = df_norm['following'].idxmax()

# Get the corresponding row for the user
user_data_highest_following = df_norm.loc[user_with_highest_following]

# Display the user ID and corresponding data
print("User with the highest 'following':", user_with_highest_following)
print("Data for the user:")
print(user_data_highest_following)

User with the highest 'following': nbeil10
Data for the user:
following       0.991434
likes           0.007654
videos_count    0.000911
Name: nbeil10, dtype: float64


In [22]:
# find user wtih highest 'likes'

user_with_highest_likes = df_norm['likes'].idxmax()

# Get the corresponding row for the user
user_data_highest_likes = df_norm.loc[user_with_highest_likes]

# Display the user ID and corresponding data
print("User with the highest 'likes':", user_with_highest_likes)
print("Data for the user:")
print(user_data_highest_likes)

User with the highest 'likes': diana_aster
Data for the user:
following       3.458205e-08
likes           9.999975e-01
videos_count    2.440916e-06
Name: diana_aster, dtype: float64


In [23]:
# find user with highest videos_count

user_with_highest_videos_count = df_norm['videos_count'].idxmax()

# Get the corresponding row for the user
user_data_highest_videos_count = df_norm.loc[user_with_highest_videos_count]

# Display the user ID and corresponding data
print("User with the highest 'videos_count':", user_with_highest_videos_count)
print("Data for the user:")
print(user_data_highest_videos_count)

User with the highest 'videos_count': __a716
Data for the user:
following       0.111111
likes           0.666667
videos_count    0.222222
Name: __a716, dtype: float64


In [24]:
from scipy.sparse import lil_matrix
import scipy.spatial.distance


In [25]:
# create user_following_map
user_following_map = dict(zip(df_norm.index, df_norm['following']))

#user with highest 'following'
target_user_id1 = 'nbeil10'

if target_user_id1 in user_following_map:
    target_user_following = user_following_map[target_user_id1]

# find cosine distances
    distances = scipy.spatial.distance.cdist(df_norm, [df_norm.loc[target_user_id1].values], metric="cosine").flatten()

    query_distances = list(zip(df_norm.index, distances))

    for similar_user_id, similar_distance in sorted(query_distances, key=lambda x: x[1], reverse=False)[:10]:
        print(similar_user_id, similar_distance, df_norm.loc[similar_user_id].sum())
    
else:
    print(f"User '{target_user_id1} not found in the user_following_map.")


nbeil10 0.0 1.0
user401012101830068 1.3338668400919573e-06 1.0
eminence713 8.992726383882932e-06 1.0
oi_ilk 3.5288340739847435e-05 1.0
bvmonmi 3.757949773008917e-05 1.0
yumi_blox_ 8.637881351369892e-05 1.0
dkr3d05 8.804738184198246e-05 1.0
gneaxb_788 0.00010610679912581578 1.0
dgueixasantoslima 0.0001791596023803521 1.0
dycrprfw88c4 0.0002074564394984879 1.0


In [26]:
# create a dataframe with most similar users to look at stats

# List of user IDs with high similarity
similar_users = [similar_user_id for similar_user_id, _ in sorted(query_distances, key=lambda x: x[1], reverse=False)[:10]]

# Create a DataFrame with data for the similar users
similar_users_df = df_norm.loc[similar_users]

# Print the DataFrame
print(similar_users_df)

                     following     likes  videos_count
account_id                                            
nbeil10               0.991434  0.007654      0.000911
user401012101830068   0.989423  0.009180      0.001397
eminence713           0.987057  0.011801      0.001142
oi_ilk                0.983865  0.015840      0.000295
bvmonmi               0.983607  0.016101      0.000293
yumi_blox_            0.978790  0.020424      0.000786
dkr3d05               0.976744  0.020349      0.002907
gneaxb_788            0.976871  0.021769      0.001361
dgueixasantoslima     0.971894  0.025857      0.002248
dycrprfw88c4          0.971588  0.027298      0.001114


In [27]:
# create user_likes_map
user_likes_map = dict(zip(df_norm.index, df_norm['likes']))

#user with highest 'likes'
target_user_id2 = 'diana_aster'


if target_user_id2 in user_likes_map:
    target_user_likes = user_likes_map[target_user_id2]

# find cosine distances
    distances = scipy.spatial.distance.cdist(df_norm, [df_norm.loc[target_user_id2].values], metric="cosine").flatten()

    query_distances = list(zip(df_norm.index, distances))

    for similar_user_id, similar_distance in sorted(query_distances, key=lambda x: x[1], reverse=False)[:10]:
        print(similar_user_id, similar_distance, df_norm.loc[similar_user_id].sum())
    
else:
    print(f"User '{target_user_id2} not found in the user_following_map.")

diana_aster 1.1102230246251565e-16 1.0
itsmenicksmithy2 3.430589146091734e-14 1.0
lionsgate 2.5990321006474915e-13 1.0
katjakrasavice 2.950972799453666e-13 1.0
biteswithlily 8.386846772623358e-12 1.0
solsenpai 1.1958656287447411e-11 1.0
duki 1.390176862514636e-11 1.0
rzdarmawangsa 1.90528703924997e-11 1.0
lyllistapper 2.459732417747773e-11 0.9999999999999999
aru.rinh 3.825539884871887e-11 1.0


In [28]:
# create a dataframe with most similar users to look at stats

# List of user IDs with high similarity
similar_users2 = [similar_user_id for similar_user_id, _ in sorted(query_distances, key=lambda x: x[1], reverse=False)[:10]]

# Create a DataFrame 
similar_users_df2 = df_norm.loc[similar_users2]

# Print the DataFrame
print(similar_users_df2)

                     following     likes  videos_count
account_id                                            
diana_aster       3.458205e-08  0.999998  2.440916e-06
itsmenicksmithy2  5.442950e-08  0.999997  2.701788e-06
lionsgate         3.728057e-07  0.999997  3.077475e-06
katjakrasavice    3.241481e-08  0.999997  3.209066e-06
biteswithlily     3.836287e-06  0.999992  3.964163e-06
solsenpai         1.905389e-06  0.999991  6.959398e-06
duki              4.999972e-06  0.999994  6.666629e-07
rzdarmawangsa     4.950452e-08  0.999991  8.613787e-06
lyllistapper      6.942679e-06  0.999989  3.654042e-06
aru.rinh          2.499967e-06  0.999987  1.083319e-05


In [31]:


# Specify  user 
target_user = 'diana_aster'  # Replace with the actual account_id

# Filter the original DataFrame 
user_row = df[df['account_id'] == target_user]

# Display the user's original data
print(user_row)

      timestamp   account_id         nickname                    biography  \
733  2023-07-30  diana_aster  💗 DIANA ASTER 💗  ЖДУ ТЕБЯ В ТЕЛЕГЕ 👇🏼💝\n\n ⠀   

     awg_engagement_rate  comment_engagement_rate  like_engagement_rate  \
733             0.010912                 0.000047              0.010866   

                bio_link  is_verified  followers  following      likes  \
733  https://t.me/dom2di         True   10500000         12  347000000   

     videos_count  create_time                   id  \
733           847          NaN  6575837798929235974   

                                            top_videos  \
733  https://www.tiktok.com/@diana_aster/video/7260...   

                                     url  \
733  https://www.tiktok.com/@diana_aster   

                                       profile_pic_url  
733  https://p16-sign-sg.tiktokcdn.com/aweme/720x72...  


In [31]:
# create user_videos_count_map
user_videos_count_map = dict(zip(df_norm.index, df_norm['videos_count']))

# user with highest 'videos count' 
target_user_id3 = '__a716'


if target_user_id3 in user_videos_count_map:
    target_user_videos_count = user_videos_count_map[target_user_id3]

# find cosine distances
    distances = scipy.spatial.distance.cdist(df_norm, [df_norm.loc[target_user_id3].values], metric="cosine").flatten()

    query_distances = list(zip(df_norm.index, distances))

    for similar_user_id, similar_distance in sorted(query_distances, key=lambda x: x[1], reverse=False)[:10]:
        print(similar_user_id, similar_distance, df_norm.loc[similar_user_id].sum())
    
else:
    print(f"User '{target_user_id3} not found in the user_following_map.")


__a716 2.220446049250313e-16 0.9999999999999999
xehgmkas 0.012537393380061657 1.0
gol.fm 0.014083000786515476 1.0
audi_uk 0.0168242058491731 1.0
fownfown4 0.02119911240162209 1.0
jordanannpetersen 0.022878474715016583 1.0
dh.02xspam 0.02395913583103193 1.0
xx.h.me0 0.02400636802620426 0.9999999999999999
lpqzxc 0.02496011621818639 1.0
gmcrath30 0.02498866132220634 1.0


In [32]:
# create a dataframe with most similar users to look at stats

# List of user IDs with high similarity
similar_users3 = [similar_user_id for similar_user_id, _ in sorted(query_distances, key=lambda x: x[1], reverse=False)[:10]]

# Create a DataFrame with data for the similar users
similar_users_df3 = df_norm.loc[similar_users3]

# Print the DataFrame
print(similar_users_df3)

                   following     likes  videos_count
account_id                                          
__a716              0.111111  0.666667      0.222222
xehgmkas            0.078431  0.784314      0.137255
gol.fm              0.049505  0.801980      0.148515
audi_uk             0.075188  0.804511      0.120301
fownfown4           0.155556  0.755556      0.088889
jordanannpetersen   0.186945  0.722119      0.090936
dh.02xspam          0.131579  0.789474      0.078947
xx.h.me0            0.094000  0.820000      0.086000
lpqzxc              0.192513  0.721925      0.085561
gmcrath30           0.167967  0.754550      0.077483


In [32]:


# Specify user 
target_user1 = '__a716' 

# Filter the original DataFrame
user_row = df[df['account_id'] == target_user1]

# Display the user's original data
print(user_row)

      timestamp account_id nickname biography  awg_engagement_rate  \
264  2023-07-27     __a716   __a716     nyeks                0.875   

     comment_engagement_rate  like_engagement_rate bio_link  is_verified  \
264                    0.125                  0.75      NaN        False   

     followers  following  likes  videos_count  create_time  \
264          4          1      6             2          NaN   

                      id                                         top_videos  \
264  6944276174970553345  https://www.tiktok.com/@__a716/video/696658635...   

                                url  \
264  https://www.tiktok.com/@__a716   

                                       profile_pic_url  
264  https://p16-sign-sg.tiktokcdn.com/aweme/720x72...  


In [33]:
# Specify user 
target_user2 = 'xehgmkas' 

# Filter the original DataFrame 
user_row = df[df['account_id'] == target_user2]

# Display the user's original data
print(user_row)

      timestamp account_id            nickname  \
285  2023-07-25   xehgmkas  иди к своей цели)☆   

                                             biography  awg_engagement_rate  \
285  https://t.me/XasenGFd ссылка на тг заходи и де...             3.142857   

     comment_engagement_rate  like_engagement_rate bio_link  is_verified  \
285                 0.285714              2.857143      NaN        False   

     followers  following  likes  videos_count  create_time  \
285          2          4     40             7          NaN   

                      id                                         top_videos  \
285  7091548516405576705  https://www.tiktok.com/@xehgmkas/video/7096169...   

                                  url  \
285  https://www.tiktok.com/@xehgmkas   

                                       profile_pic_url  
285  https://p16-sign-sg.tiktokcdn.com/aweme/720x72...  
